# Stitching images using translation only
Leanne Friedrich 5/22/21

## initialization

In [1]:
%load_ext autoreload
%autoreload 2
LOGGERDEFINED = False

In [2]:
import os
import cv2
import sys
sys.path.append("../")
sys.path.append("../../")
from py.tools.config import cfg
import py.tools.logs as logs
import py.pic_stitch_bas as sb
from matplotlib import pyplot as plt
LOGGERDEFINED = logs.openLog('pic_stitch_bas.ipynb', LOGGERDEFINED, level='DEBUG', exportLog=False)

## test stitching

In [77]:
run ../../tests/test_stitch_bas

INFO: 
xs
-----
INFO: 
horiz
-----
INFO: 
vert
-----


.

INFO: 
xs
-----
INFO: 
horiz
-----
INFO: 
vert
-----


.

INFO: 
xs
-----
INFO: Wrote image I_PDMSM7.5_S_3.50_210630\I_PDMSM7.5_S_3.50_210630_xs1_1_00.png
INFO: 
horiz
-----
INFO: Wrote image I_PDMSM7.5_S_3.50_210630\I_PDMSM7.5_S_3.50_210630_horizfull_0.273_00.png
INFO: Test stitch passed: //cfs2e.nist.gov/642/internal/3D Printing Lab/Shopbot/videos/singleLines\PDMSM LapRD\I_PDMSM7.5_S_3.50\I_PDMSM7.5_S_3.50_210630
INFO: 
vert
-----
INFO: Wrote image I_PDMSM7.5_S_3.50_210630\I_PDMSM7.5_S_3.50_210630_vert1_0.429_00.png


...

.

INFO: 
xs
-----
INFO: Wrote image I_PEG5_S_2.50_211006\I_PEG5_S_2.50_211006_xs1_1_00.png
INFO: 
horiz
-----
INFO: Wrote image I_PEG5_S_2.50_211006\I_PEG5_S_2.50_211006_horizfull_0.3_00.png
INFO: 
vert
-----
INFO: Wrote image I_PEG5_S_2.50_211006\I_PEG5_S_2.50_211006_vert1_0.333_00.png


.

INFO: 
xs
-----
INFO: Wrote image I_PEG7.5_S_4.00_211006\I_PEG7.5_S_4.00_211006_xs1_1_00.png
INFO: 
horiz
-----
INFO: Wrote image I_PEG7.5_S_4.00_211006\I_PEG7.5_S_4.00_211006_horizfull_0.3_00.png
INFO: 
vert
-----
INFO: Wrote image I_PEG7.5_S_4.00_211006\I_PEG7.5_S_4.00_211006_vert1_0.333_00.png


.
----------------------------------------------------------------------
Ran 12 tests in 153.992s

OK


# stitch images

In [44]:
fl = sb.stitchSorterSingle(os.path.join(cfg.path.server, r'singleLines\PEGDA LapRD\I_PEG7.5_S_4.00\I_PEG7.5_S_4.00_210630'))
fl.printGroups()

INFO: 
horiz
-----
INFO: horizfull: Stitch: I_PEG7.5_S_4.00_210630_horizfull_0.273_00.png
	Stills:singleLinesPics4_Basler camera_I_PEG7.5_S_4.00_210630_185435.png
[['35' '52' '09' '26' '44' '01']
 ['36' '53' '10' '28' '45' '02']
 ['37' '55' '12' '29' '46' '03']
 ['39' '56' '13' '30' '47' '04']
 ['40' '57' '14' '31' '48' '05']
 ['41' '58' '15' '32' '49' '06']
 ['42' '59' '16' '33' '51' '08']
 ['43' '00' '17' '35' '52' '09']
 ['44' '02' '19' '36' '53' '10']
 ['46' '03' '20' '37' '54' '11']
 ['51' '08' '25' '42' '59' '']]
INFO: 
vert
-----
INFO: vert1: Stitch: I_PEG7.5_S_4.00_210630_vert1_0.429_00.png
	Stills:singleLinesPics4_Basler camera_I_PEG7.5_S_4.00_210630_185620.png
[['20']
 ['21']
 ['22']
 ['23']
 ['25']
 ['26']
 ['27']]
INFO: vert2: Stitch: I_PEG7.5_S_4.00_210630_vert2_0.429_00.png
	Stills:singleLinesPics4_Basler camera_I_PEG7.5_S_4.00_210630_185631.png
[['31']
 ['32']
 ['33']
 ['34']
 ['36']
 ['37']
 ['38']]
INFO: vert3: Stitch: I_PEG7.5_S_4.00_210630_vert3_0.429_00.png
	Stills:

In [27]:
fl = sb.stitchSorterTriple(os.path.join(cfg.path.server, r'tripleLines\mineral_LapRD_visc\I_M9_S_3.00\I_M9_S_3.00_220426'))
fl.printGroups()

INFO: 
HIB
-----
INFO: 
HIPh
-----
INFO: 
HIPxs
-----
INFO: HIPxs_0.500_0: Stitch: tripleLinesXS_+y_0.500_HIPxs_0_stitch_1_I_M9_S_3.00_220426_191303_5.png
	Stills:tripleLinesXSPics_Basler camera_I_M9_S_3.00_220426_191303_5.png

INFO: HIPxs_0.500_1: Stitch: tripleLinesXS_+y_0.500_HIPxs_1_stitch_1_I_M9_S_3.00_220426_191306_0.png
	Stills:tripleLinesXSPics_Basler camera_I_M9_S_3.00_220426_191306_0.png

INFO: HIPxs_0.500_2: Stitch: tripleLinesXS_+y_0.500_HIPxs_2_stitch_1_I_M9_S_3.00_220426_191308_5.png
	Stills:tripleLinesXSPics_Basler camera_I_M9_S_3.00_220426_191308_5.png

INFO: HIPxs_0.500_3: Stitch: tripleLinesXS_+y_0.500_HIPxs_3_stitch_1_I_M9_S_3.00_220426_191311_1.png
	Stills:tripleLinesXSPics_Basler camera_I_M9_S_3.00_220426_191311_1.png

INFO: HIPxs_0.625_0: Stitch: tripleLinesXS_+y_0.625_HIPxs_0_stitch_1_I_M9_S_3.00_220426_191316_8.png
	Stills:tripleLinesXSPics_Basler camera_I_M9_S_3.00_220426_191316_8.png

INFO: HIPxs_0.625_1: Stitch: tripleLinesXS_+y_0.625_HIPxs_1_stitch_1_I_M9_S_

In [86]:
sb.stitchRecursive(os.path.join(cfg.path.server, r'tripleLines\\LapRD_LapRD_visc\I_3.00_S_2.25\I_3.00_S_2.25_220428'), overwrite=False)

INFO: 
--------------
--------------
//cfs2e.nist.gov/642/internal/3D Printing Lab/Shopbot/videos/tripleLines\LapRD_LapRD_visc\I_3.00_S_2.25\I_3.00_S_2.25_220428
-------
INFO: 
HOB
-----
INFO: Wrote image crossDoubleHoriz_0.5_0.625\crossDoubleHoriz_0.5_0.625_HOB_1_stitch_1_I_3.00_S_2.25_220428_101033_6.png
INFO: Wrote image crossDoubleHoriz_0.5_0.750\crossDoubleHoriz_0.5_0.750_HOB_2_stitch_1_I_3.00_S_2.25_220428_101039_8.png
INFO: Wrote image crossDoubleHoriz_0.5_0.875\crossDoubleHoriz_0.5_0.875_HOB_3_stitch_1_I_3.00_S_2.25_220428_101046_2.png
INFO: Wrote image crossDoubleHoriz_0.5_1.000\crossDoubleHoriz_0.5_1.000_HOB_4_stitch_1_I_3.00_S_2.25_220428_101052_5.png
INFO: Wrote image crossDoubleHoriz_0.5_1.250\crossDoubleHoriz_0.5_1.250_HOB_5_stitch_1_I_3.00_S_2.25_220428_101058_9.png
INFO: 
HOC
-----
INFO: Wrote image crossDoubleHoriz_0_0.625\crossDoubleHoriz_0_0.625_HOC_1_stitch_1_I_3.00_S_2.25_220428_101120_5.png
INFO: Wrote image crossDoubleHoriz_0_0.750\crossDoubleHoriz_0_0.750_HOC_2_

KeyboardInterrupt: 

In [27]:
sb.stitchRecursive(os.path.join(cfg.path.server, r'tripleLines\mineral_LapRD_visc\I_M9_S_3.00\I_M9_S_3.00_220426'), overwrite=True)


HIB
-----

HIPh
-----

HIPxs
-----
HIPxs_0.500_0: Stitch: tripleLinesXS_+y_0.500_HIPxs_0_stitch_1_I_M9_S_3.00_220426_191303_5.png
	Stills:tripleLinesXSPics_Basler camera_I_M9_S_3.00_220426_191303_5.png
[['191303_5']]
HIPxs_0.500_1: Stitch: tripleLinesXS_+y_0.500_HIPxs_1_stitch_1_I_M9_S_3.00_220426_191306_0.png
	Stills:tripleLinesXSPics_Basler camera_I_M9_S_3.00_220426_191306_0.png
[['191306_0']]
HIPxs_0.500_2: Stitch: tripleLinesXS_+y_0.500_HIPxs_2_stitch_1_I_M9_S_3.00_220426_191308_5.png
	Stills:tripleLinesXSPics_Basler camera_I_M9_S_3.00_220426_191308_5.png
[['191308_5']]
HIPxs_0.500_3: Stitch: tripleLinesXS_+y_0.500_HIPxs_3_stitch_1_I_M9_S_3.00_220426_191311_1.png
	Stills:tripleLinesXSPics_Basler camera_I_M9_S_3.00_220426_191311_1.png
[['191311_1']]
HIPxs_0.625_0: Stitch: tripleLinesXS_+y_0.625_HIPxs_0_stitch_1_I_M9_S_3.00_220426_191316_8.png
	Stills:tripleLinesXSPics_Basler camera_I_M9_S_3.00_220426_191316_8.png
[['191316_8']]
HIPxs_0.625_1: Stitch: tripleLinesXS_+y_0.625_HIPxs_1_

KeyboardInterrupt: 

In [64]:
import fileHandling as fh

def sortStitch(printFolder:str, real:str):
    good = []
    bad = []
    pfd = fh.printFileDict(printFolder)
    for s in pfd.stitch:
        if real in s:
            good.append(s)
        else:
            bad.append(s)
    return good,bad

def sorts(fHOB:str, fHOC:str, real:str):
    goodHOB,badHOB = sortStitch(fHOB, real)
    if len(goodHOB)==0:
        goodHOC1,badHOC1 = sortStitch(fHOC, real)
        if len(goodHOC1)>0:
            bn1 = os.path.basename(fHOB)
            bn2 = os.path.basename(fHOC)
            oldname = goodHOC1[0]
            newname = goodHOC1[0].replace(bn2, bn1)
            os.rename(oldname, newname)
            print(f'Renamed {fh.twoBN(oldname)} to {fh.twoBN(newname)}')
    return badHOB


def switchCB(folder:str):
    if not fh.isSubFolder(folder):
        return
    fl = sb.fileList(folder)
    for i,fHOB in fl.HOBfolders.items():
        fHOC = fl.HOCfolders[i]
        badHOB = sorts(fHOB, fHOC, 'HOB')
        badHOC = sorts(fHOC, fHOB, 'HOC')
        goodHOB,badHOB = sortStitch(fHOB, 'HOB')
        if len(badHOB)>0:
            for file in badHOB:
                os.remove(file)
        goodHOC,badHOC = sortStitch(fHOC, 'HOC')
        if len(badHOC)>0:
            for file in badHOC:
                os.remove(file)
    for i,fVB in fl.VBfolders.items():
        fVC = fl.VCfolders[i]
        badVB = sorts(fVB, fVC, 'VB')
        badVC = sorts(fVC, fVB, 'VC')

topFolder = r'\\cfs2e.nist.gov\642\internal\3D Printing Lab\Shopbot\videos\tripleLines'
for typeFolder0 in os.listdir(topFolder):
    typeFolder = os.path.join(topFolder, typeFolder0)
    for sampleFolder0 in os.listdir(typeFolder):
        sampleFolder = os.path.join(typeFolder, sampleFolder0)
        for subFolder0 in os.listdir(sampleFolder):
            subFolder = os.path.join(sampleFolder, subFolder0)
            try:
                switchCB(subFolder)
            except:
                print(subFolder)
                pass

\\cfs2e.nist.gov\642\internal\3D Printing Lab\Shopbot\videos\tripleLines\PDMSS_LapRD_visc\I_PDMSS12.5_S_4.00\I_PDMSS12.5_S_4.00_220503
